In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

import tifffile
import sys
from astropy.visualization import AsymmetricPercentileInterval, LogStretch, MinMaxInterval


ModuleNotFoundError: No module named 'tifffile'

In [ ]:
WORKDIR='/home/daniel/gdrive/EPFL/2019-2020/MachineLearning/Project/gravitational_lens_ml'
SRC = os.path.join(WORKDIR, 'src')
DATA = os.path.join(WORKDIR,'data')
RESULTS = os.path.join(WORKDIR, 'results')
TRAIN = os.path.join(DATA, 'datapack2.0train/Public')
TEST = os.path.join(DATA, 'datapack2.0test/Public')
TRAIN_MULTIBAND = os.path.join(DATA, 'train_multiband')
TEST_MULTIBAND = os.path.join(DATA, 'test_multiband')

In [2]:
image_catalog = pd.read_csv(os.path.join(DATA, 'catalog/image_catalog2.0train.csv'), comment='#', index_col=0)
print(image_catalog.shape)
display(image_catalog.isna().sum(axis=0))
# No effective magnification for 11182 records.

NameError: name 'DATA' is not defined

In [3]:
band = 'EUC_VIS'
def get_file_id(filename, delimiters = '_|\.|-'):
    id_ = [int(s) for s in re.split(delimiters, filename) if s.isdigit()][0]
    return id_
file_id_train = np.array([get_file_id(f) for f in os.listdir(os.path.join(TRAIN, band))], dtype=int)
file_id_test = np.array([get_file_id(f) for f in os.listdir(os.path.join(TEST, band))], dtype=int)
def check_existing_files(band, set_):
    for ID in image_catalog.ID:
        if not os.path.isfile(os.path.join(set_, band,'image%s-%i.fits'%(band, ID))):
            print('File image%s-%i.fits does not exist in set.'%(band, ID))
missing_img =  np.setdiff1d(image_catalog.ID.values, file_id_train, assume_unique=False)
print(file_id_train.shape)
print(missing_img)

NameError: name 'TRAIN' is not defined

In [ ]:
image_catalog['is_lens'] = (image_catalog['mag_lens'] > 1.2) & (image_catalog['n_sources'] != 0)
print('Number of lenses: %i'%image_catalog['is_lens'].sum())

In [ ]:
image_catalog['img_exists'] = True
image_catalog['img_exists'].loc[image_catalog['ID'].isin(missing_img)] = False

In [ ]:
image_catalog = image_catalog.drop_duplicates('ID')
image_catalog[['ID',
               'is_lens']][image_catalog['img_exists']].to_csv(os.path.join(
                   RESULTS, 'lens_id_labels.csv'),
                                                               index=False)

In [ ]:
import aplpy 
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp

In [ ]:
def get_image_filename_from_id(id_, band, set_ = TRAIN, ext = 'fits'):
    fname = os.path.join(set_, '{0}/image{0}-{1}.{2}'.format(band, id_, ext))
    return fname
def plot_all_bands_from_id(id_):
    fig = plt.figure(figsize=(20, 6))
    bands = ['EUC_VIS', 'EUC_H', 'EUC_Y', 'EUC_J']
    fitsfigs = []
    data = []
    for i, band in enumerate(bands):
        with fits.open(get_image_filename_from_id(id_, band)) as hdu:
            data.append(hdu[0].data)
        fitsfigs.append(aplpy.FITSFigure(data[i], figure=fig, subplot=(1,4,i+1)))
        fitsfigs[i].show_colorscale()
        fitsfigs[i].set_theme('preety')
        fitsfigs[i].set_title(band)
        fitsfigs[i].add_colorbar()
    fig.tight_layout() 
with fits.open(get_image_filename_from_id(290000, 'EUC_VIS')) as hdu1:
    with fits.open(get_image_filename_from_id(290000, 'EUC_H')) as hdu2:
        data1 = hdu1[0].data
        data2 = hdu2[0].data
        data2_reprojected, data2_footprint = reproject_interp(hdu2[0], hdu1[0].header)


In [ ]:
plot_all_bands_from_id(270610)

# Combine bands into `tiff` and test

In [ ]:
def build_image(id_, set_, bands = ['EUC_VIS', 'EUC_H', 'EUC_J', 'EUC_Y'], img_size=200, scale = 100):
    tables = []
    data = np.empty((img_size, img_size, len(bands)))
    for i, band in enumerate(bands):
        tables.append(fits.open(get_image_filename_from_id(id_, band)))
        if band != 'EUC_VIS':
            band_data, data_footprint = reproject_interp(tables[i][0], tables[0][0].header)
        else:
            band_data = tables[0][0].data
        band_data[np.isnan(band_data)] = 0.
        interval = AsymmetricPercentileInterval(0.25, 99.75, n_samples=10000)
        vmin, vmax = interval.get_limits(band_data)
        stretch = MinMaxInterval() +  LogStretch()
        data[:,:,i] = stretch(((np.clip(band_data, vmin*(-0.7), vmax))/(vmax)))
    for t in tables:
        t.close()
    return data.astype(np.float32)


In [ ]:
img = build_image(270210, TRAIN)
print(sys.getsizeof(img))
for i in range(4):
    plt.figure()
    print(np.min(img[:,:,i]), np.max(img[:,:,i]))
    a = plt.imshow((img[:,:,i]))
print(image_catalog.loc[image_catalog['ID']==270210]['is_lens'])

# Compare preprocessing with Log + Norm

In [ ]:
def build_image_noprocess(id_, set_, bands = ['EUC_VIS', 'EUC_H', 'EUC_J', 'EUC_Y'], img_size=200, scale = 100):
    tables = []
    data = np.empty((img_size, img_size, len(bands)))
    for i, band in enumerate(bands):
        tables.append(fits.open(get_image_filename_from_id(id_, band)))
        if band != 'EUC_VIS':
            band_data, data_footprint = reproject_interp(tables[i][0], tables[0][0].header)
        else:
            band_data = tables[0][0].data
        band_data[np.isnan(band_data)] = 0.
        norm =  LogStretch()
        data[:,:,i] = norm(band_data)
    for t in tables:
        t.close()
    return data.astype(np.float32)


In [ ]:
test_id = 270210
img = build_image(test_id, TRAIN)
img_nopr = build_image_noprocess(test_id, TRAIN)
def plot_slice(image, slice_ratio, axis=0):
    '''Plot a single slice of a picture.
    
    image: Image to plot.
    slice_ratio: float from 0 to 1 where 0 is the top slice, and 1 is the last slice.
    axis: 0 or 1 to show horizontal or vertical slices'''
    
    index = int(image.shape[axis] * slice_ratio)
    slice_ = image[index]
    plt.plot(slice_)


In [ ]:
plot_slice(img, 0.5)
plt.figure()
plot_slice(img_nopr, 0.5)

In [ ]:
def plot_slice_stack(image, axis=0):
    '''Plot a single slice of a picture.
    
    image: Image to plot.'''
    
    slice_ = np.sum(image, axis=axis)
    print(slice_.shape)
    plt.plot(slice_)

In [ ]:
axis = 0
plot_slice_stack(img, axis = axis)
plt.figure()
plot_slice_stack(img_nopr, axis = axis)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)


# PCA

In [ ]:
img = tifffile.imread(DATA+'/train_multiband/image_299896_multiband.tiff')
def flatten_by_channel(original_image):
    """preprocess the image."""
    n_channels = original_image.shape[-1]
    processed_image = original_image.reshape(-1,n_channels)
    return processed_image.astype(np.float32) #Overflow if using unsigned int
flat_img = flatten_by_channel(img)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pca = PCA(whiten=False)
std_flat_img = scaler.fit_transform(flat_img)
pca.fit(std_flat_img)

In [ ]:
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)

In [ ]:
pca_flat_img = pca.fit_transform(std_flat_img)
pca_image = pca_flat_img.reshape((img.shape[0], img.shape[1], -1))

In [ ]:
f, ax = plt.subplots(2, 2, figsize = (7, 7))
for i, a in enumerate(ax.ravel()):
    a.imshow(pca_image[:,:,i])

## Covariance Matrix between colors
See, that it is almost diagonal. Not surprising that PC's are almost the same as the priginal colors.

In [ ]:
plt.imshow(np.cov(flat_img.T))

# Test  CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, Iterator
from sklearn.model_selection import train_test_split

In [ ]:
# From Tensorflow examples
batch_size = 24
epochs = 15
IMG_HEIGHT = 200
IMG_WIDTH = 200


In [ ]:
lens_df = pd.read_csv(os.path.join(RESULTS, 'lens_id_labels.csv'), index_col = 0)
def build_generator_dataframe(id_label_df, directory = TRAIN_MULTIBAND):
    files = os.listdir(directory)
    ids = [
        get_file_id(filename)
        for filename in files
    ]
    df = pd.DataFrame()
    df['filenames'] = files
    df['labels'] = id_label_df.loc[ids, 'is_lens'].values.astype(int)
    df['ID'] = ids
    return df
local_test_df = build_generator_dataframe(lens_df, TRAIN_MULTIBAND)
train_df, val_df = train_test_split(local_test_df, test_size=0.1, random_state=42)
total_train = len(train_df)
total_val = len(val_df)
display(local_test_df.head())

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,4)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
class TiffImageDataGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super(TiffImageDataGenerator, self).__init__(*args, **kwargs)
    def get_input(self, path):
        img = tifffile.imread(path)
        return img
    def image_generator_dataframe(self,dataframe,
                              directory='',
                              x_col='filename',
                              y_col='class',
                              batch_size=64,
                              validation=False):
        files = dataframe[x_col].values
        while True:
            # Select files (paths/indices) for the batch
            batch_paths = np.random.choice(a=files, size=batch_size)
            batch_input = []
            batch_output = []

            # Read in each input, perform preprocessing and get labels
            for input_path in batch_paths:
                input = self.get_input(os.path.join(directory, input_path))
                output = dataframe[dataframe[x_col] == input_path][y_col].values[0]
                if self.preprocessing_function:
                    input = self.preprocessing_function(input)
                if not validation:
                    input = self.random_transform(input)
                batch_input += [input]
                batch_output += [output]
            # Return a tuple of (input,output) to feed the network
            batch_x = np.array(batch_input)
            batch_y = np.array(batch_output)

            yield (batch_x, batch_y)
    

In [ ]:
image_data_gen_train = TiffImageDataGenerator(featurewise_center=False,
                                          samplewise_center=False,
                                          featurewise_std_normalization=False,
                                          samplewise_std_normalization=False,
                                          zca_whitening=False,
                                          zca_epsilon=1e-06,
                                          rotation_range=90,
                                          width_shift_range=0.0,
                                          height_shift_range=0.0,
                                          brightness_range=(0.8, 1.1),
                                          shear_range=0.0,
                                          zoom_range=(0.9, 1.1),
                                          channel_shift_range=0.0,
                                          fill_mode='wrap',
                                          cval=0.0,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          rescale=None,
                                          preprocessing_function=None,
                                          data_format='channels_last',
                                          dtype='float32')
image_data_gen_val = TiffImageDataGenerator(dtype='float32')

In [ ]:
train_data_gen = image_data_gen_train.image_generator_dataframe(train_df,
                                  directory=TRAIN_MULTIBAND,
                                  x_col='filenames',
                                 y_col='labels', batch_size = 1, validation=False)
val_data_gen = image_data_gen_val.image_generator_dataframe(train_df,
                                  directory=TRAIN_MULTIBAND,
                                  x_col='filenames',
                                 y_col='labels', batch_size = 1, validation=True)
fig, ax = plt.subplots(1,6, figsize = (15, 2.5))
for a in ax.ravel():
    img, label = next(train_data_gen)
    a.imshow(img[0][:,:,2])
    a.set_xlabel(label[0])


# Augment non-lens sample


In [ ]:
no_lens_df = local_test_df[local_test_df['labels'] == 0]
display(no_lens_df.head())
print('Non-lens original sample size: %s'%len(no_lens_df))
print('Non-lens original sample size: %s'%)
print('Total original sample size: %s'%len(lens_df))

In [ ]:
augment_nolens = TiffImageDataGenerator(featurewise_center=False,
                                          samplewise_center=False,
                                          featurewise_std_normalization=False,
                                          samplewise_std_normalization=False,
                                          zca_whitening=False,
                                          zca_epsilon=1e-06,
                                          rotation_range=20,
                                          width_shift_range=0.0,
                                          height_shift_range=0.0,
                                          brightness_range=(0.8, 1.1),
                                          shear_range=0.0,
                                          zoom_range=(0.9, 1),
                                          channel_shift_range=0.0,
                                          fill_mode='wrap',
                                          cval=0.0,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          rescale=None,
                                          preprocessing_function=None,
                                          data_format='channels_last',
                                          dtype='float32')
augment_nolens_gen = augment_nolens.image_generator_dataframe(no_lens_df,
                                  directory=TRAIN_MULTIBAND,
                                  x_col='filenames',
                                 y_col='labels', batch_size = 1, validation=False)


In [ ]:
i = 8
fig, ax = plt.subplots(1,i, figsize = (2.5*i, 2.5))
for a in ax.ravel():
    img, label = next(augment_nolens_gen)
    a.imshow(img[0][:,:,3])
    a.set_xlabel(label[0])
plt.tight_layout()

# Load saved model

In [ ]:
new_model = tf.keras.models.load_model(os.path.join(RESULTS, 'simple_cnn.h5'))

# Check its architecture
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate_generator(val_data_gen, steps = 1000, use_multiprocessing=False, verbose  = 1)

In [ ]:
predictions = new_model.predict_generator(augment_nolens_gen, steps = 10)
print(predictions)

In [ ]:
next(augment_nolens_gen)[0].shape

In [ ]:
random_prediction = new_model.predict(np.random.random(200*200*4).reshape(200, 200, 4)[None, :, :, :])
print('The prediction on random noise is: %.2f'%random_prediction)

In [ ]:
help(tifffile.imwrite)